In [5]:
# Install required libraries
# !pip install yfinance pandas numpy scipy matplotlib seaborn

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [7]:
def get_nasdaq100_symbols():
    nasdaq100 = pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')[4]
    return nasdaq100['Ticker'].tolist()

symbols = get_nasdaq100_symbols()

In [11]:
def get_stock_data(symbol):
    stock = yf.Ticker(symbol)
    
    try:
        info = stock.info
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        cash_flow = stock.cashflow
        
        # Enterprise Value
        ev = info.get('enterpriseValue', np.nan)
        
        # EBIT (try different methods)
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit', financials.columns[0]]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income', financials.columns[0]]
        else:
            # Estimate EBIT
            revenue = financials.loc['Total Revenue', financials.columns[0]]
            cogs = financials.loc['Cost Of Revenue', financials.columns[0]] if 'Cost Of Revenue' in financials.index else 0
            operating_expenses = financials.loc['Total Operating Expenses', financials.columns[0]] if 'Total Operating Expenses' in financials.index else 0
            ebit = revenue - cogs - operating_expenses
        
        # Acquirer's Multiple
        acquirers_multiple = ev / ebit if ev and ebit and ebit != 0 else np.nan
        
        # ROIC
        total_assets = balance_sheet.loc['Total Assets', balance_sheet.columns[0]]
        current_liabilities = balance_sheet.loc['Total Current Liabilities', balance_sheet.columns[0]] if 'Total Current Liabilities' in balance_sheet.index else 0
        invested_capital = total_assets - current_liabilities
        roic = ebit / invested_capital if invested_capital and invested_capital != 0 else np.nan
        
        # Revenue Growth
        if len(financials.columns) >= 4:
            revenue_recent = financials.loc['Total Revenue', financials.columns[0]]
            revenue_past = financials.loc['Total Revenue', financials.columns[3]]
            revenue_growth = (revenue_recent / revenue_past) ** (1/3) - 1 if revenue_past != 0 else np.nan
        else:
            revenue_growth = np.nan
        
        # Simplified Altman Z-Score
        retained_earnings = balance_sheet.loc['Retained Earnings', balance_sheet.columns[0]] if 'Retained Earnings' in balance_sheet.index else 0
        current_assets = balance_sheet.loc['Total Current Assets', balance_sheet.columns[0]] if 'Total Current Assets' in balance_sheet.index else 0
        working_capital = current_assets - current_liabilities
        z_score = (1.2 * working_capital / total_assets) + (1.4 * retained_earnings / total_assets) + (3.3 * ebit / total_assets) if total_assets != 0 else np.nan
        
        # RSI (14-day)
        history = stock.history(period="1mo")
        delta = history['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs.iloc[-1])) if not rs.empty and rs.iloc[-1] != 0 else np.nan
        
        beta = info.get('beta', np.nan)
        
        return pd.Series({
            'Symbol': symbol,
            'Acquirers_Multiple': acquirers_multiple,
            'ROIC': roic,
            'Revenue_Growth': revenue_growth,
            'Z_Score': z_score,
            'RSI': rsi,
            'Beta': beta
        })
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        return pd.Series({
            'Symbol': symbol,
            'Acquirers_Multiple': np.nan,
            'ROIC': np.nan,
            'Revenue_Growth': np.nan,
            'Z_Score': np.nan,
            'RSI': np.nan,
            'Beta': np.nan
        })

# Fetch data for all symbols
data = pd.DataFrame([get_stock_data(symbol) for symbol in symbols])

# Save data as csv
data.to_csv('stock_data.csv', index=False)

In [13]:
data=pd.read_csv('stock_data.csv')
data.head()

,Symbol,Acquirers_Multiple,ROIC,Revenue_Growth,Z_Score,RSI,Beta
0,ADBE,35.057655,0.223312,0.146826,2.304624,41.647898,1.298
1,ABNB,41.847183,0.073529,0.431853,0.010385,18.519885,1.189
2,GOOGL,22.786645,0.209480,0.189751,1.426253,30.679837,1.046
3,GOOG,23.051232,0.209480,0.189751,1.426253,30.321446,1.046
4,AMZN,49.085938,0.069815,0.141867,0.531732,33.288317,1.155


In [14]:
def calculate_score(data):
    # Calculate percentile ranks
    data['Value_Rank'] = data['Acquirers_Multiple'].rank(ascending=False, pct=True)
    data['Quality_Rank'] = data['ROIC'].rank(pct=True)
    data['Growth_Rank'] = data['Revenue_Growth'].rank(pct=True)
    data['Financial_Health_Rank'] = data['Z_Score'].rank(pct=True)
    data['Sentiment_Rank'] = (50 - abs(data['RSI'] - 50)).rank(pct=True)
    
    # Calculate composite score
    data['Composite_Score'] = (
        0.3 * data['Value_Rank'] +
        0.25 * data['Quality_Rank'] +
        0.2 * data['Growth_Rank'] +
        0.15 * data['Financial_Health_Rank'] +
        0.1 * data['Sentiment_Rank']
    )
    
    # Apply margin of safety
    data['Final_Score'] = data['Composite_Score'] * (1 - (data['Beta'] / 2))
    
    return data

# Calculate scores
scored_data = calculate_score(data)

# Rank stocks
final_ranking = scored_data.sort_values('Final_Score', ascending=False).reset_index(drop=True)

In [17]:
# Save to csv
from datetime import datetime
final_ranking.to_csv(f'final_ranking_{datetime.now().strftime("%Y%m%d")}.csv', index=False)

In [18]:
print(final_ranking[['Symbol', 'Final_Score']].head(10))

  Symbol  Final_Score
0    PDD     0.546994
1    AZN     0.515964
2   REGN     0.498257
3   GILD     0.480641
4   VRTX     0.462095
5   BIIB     0.419189
6    PEP     0.402294
7   CCEP     0.391050
8   PCAR     0.361724
9   ODFL     0.340851
